# LulaSafe API

This tutorial will guide you step by step on how to use the LulaSafe GraphQL API.

In [ ]:
%pip install gql[all]
%pip install requests

In [ ]:
import json
import os
from pprint import pp

import requests

## Authentication

> **Notice**
>
> Until we add support for the OpenID Connect client credentials flow, we need to perform some custom token retrieving actions.

### 1. Read your credentials
> **Important**
>
> Create an [`appsecrets.json`](../appsecrets.json) file in the root of this repo, and put your credentials into it, like so:
>
> ``` JSON
> {
>     "ClientId": "< Your Lula login >",
>     "ClientSecret": "< Your Lula password >"
> }
> 

In [ ]:
class LulaSafeOptions():
    def __init__(self, client_id, client_secret, base_url):
        self.client_id = client_id
        self.client_secret = client_secret
        self.base_url = base_url

    def __str__(self):
        return f"base_url: {self.base_url}{os.linesep}client_id: {self.client_id}"
        
    @staticmethod
    def read_from_config(appsecrets_path, appsettings_path):
        with open(appsecrets_path) as app_secrets:
            secrets_object = json.load(app_secrets)
        
        with open(appsettings_path) as app_settings:
            settings_object = json.load(app_settings)
        
        return LulaSafeOptions(secrets_object["ClientId"], secrets_object["ClientSecret"], settings_object["BaseUrl"])
    
    @staticmethod
    def read_from_default_config():
        return LulaSafeOptions.read_from_config("../appsecrets.json", "../appsettings.json")

lula_safe_config = LulaSafeOptions.read_from_default_config()
print(lula_safe_config)

In [ ]:
# Lula PATHs
init_session_path = "v1/login/initialize"
submit_login_path = "v1/login/submit"

### Session concept

If you want to use the API from a client side application (e.g. from web browser or mobile app), you must first establish a short-lived session from your back-end.
This will get you a bearer token, which you will then use to get a `sessionId`.  This `sessionId` is what you will use for making calls from your client side code.

### 1. Initialize the session flow

In [ ]:
response = requests.get(url=f"{lula_safe_config.base_url}/{init_session_path}").json()

flow_id = response.get("id")
if flow_id:
    print(f"FlowId '{flow_id}'")
else:
    print("Failed to find Flow ID in the response:")
    pp(response)

### 2. Get a bearer token

In [ ]:
request_data = {
    "method": "password",
    "identifier": lula_safe_config.client_id,
    "password": lula_safe_config.client_secret
}
full_url = f"{lula_safe_config.base_url}/{submit_login_path}?flow={flow_id}"

json_request = json.dumps(request_data).encode('utf-8')

response = requests.post(url=full_url, data=json_request, headers= {'Content-Type': 'application/json; charset=utf-8'}).json()

bearer_token = response.get("session_token")
if bearer_token:
    print(f"Bearer Token '{bearer_token}'")
else:
    print("Failed to find Bearer Token in the reponse:")
    pp(response)

### 3. Get a session token using the bearer token

Use a bearer token in the `Authorization` header to establish a session.

In [ ]:
lula_session_endpoint = f"{lula_safe_config.base_url}/risk/v0.1-beta1/sessions"

response = requests.post(url=lula_session_endpoint, headers={'Authorization': f"Bearer {bearer_token}"}).json()

session_id = response.get("sessionId")
if session_id:
    print(f"SessionId '{session_id}'")
else:
    print("Failed to find Session ID in the reponse:")
    pp(response)

Once you have a `sessionId` you, can pass it instead of the bearer token. Use it in your front-end or mobile app so you aren't exposing the bearer token there.

## GraphQL client usage
Let's define a function to read an operation from a `gql` file.

In [ ]:
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport

lulasafe_graphql_url = f"{lula_safe_config.base_url}/risk/graphql"
transport = AIOHTTPTransport(lulasafe_graphql_url, headers= {'session-id': session_id})

def read_operation(name: str):
    with open(f"../graphql/{name}", "r") as gql_file:
        return gql_file.read()

### Driver Assessment
> **Important**
>
>  Store the assessment Id on your back-end if you wish to retrieve the result again later.

Collect driver data and request an assessment for that driver using the [CheckInsuranceAndRequestVehicles.gql](../graphql/CheckInsuranceAndRequestVehicles.gql) mutation.

In [ ]:
assessee = {
    "firstName": "NOAH",
    "lastName": "RUSSAW",
    "middleName": "",
    "dateOfBirth": "1962-8-1",
    "phone": "",
    "email": ""
}

address_request = {
    "line1": "8340 BUNCHE DR",
    "line2": "",
    "zipCode": "75243",
    "country": "US",
    "city": "DALLAS",
    "state": "TX"
}

async def create_driver_assessment():
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
        parse_results=True
    ) as client:
        assess_mutation = gql(read_operation("CheckInsuranceAndRequestVehicles.gql"))
        params = {"assessee": assessee, "address": address_request}     
        response = await client.execute(assess_mutation, variable_values=params)
        driver_assessment_id = response.get("assess", {}).get("id")

        if driver_assessment_id:
            return driver_assessment_id
        else:
            print("Can't find Driver Assessment ID in response:")
            pp(response)
        

driver_assessment_id = await create_driver_assessment()
f"Assessment query ID returned from server: '{driver_assessment_id}'"

### Getting assessment results later
Get any previous assessment results by assessment Id using the [RetrieveInsuranceAndVehiclesResult.gql](../graphql/RetrieveInsuranceAndVehiclesResult.gql) query.

In [ ]:
async def get_assessment_results():
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
        parse_results=False
    ) as client:
        assessment_query = gql(read_operation("RetrieveInsuranceAndVehiclesResult.gql"))
        params = {"id": driver_assessment_id}
        response = await client.execute(assessment_query, variable_values=params)
        print("Assessment Results:")
        pp(response)

await get_assessment_results()